In [1]:
import pandas as pd
import btyd as b # https://btyd.readthedocs.io/

PATH = 'data\\wallet_urfu.parquet.gzip'

In [ ]:
# partner — идентификатор клиента
# rep_date — дата пополнения. Запись в кошелек выполняется ежедневно. Нулевые пополнения не записываются. Если у клиента поступления в некоторую дату нет, то запись не производится.
# monetary — сумма пополнения. Значения приведены к интервалу 0 — 1.

In [4]:
data = pd.read_parquet(PATH)
data

,partner,rep_date,monetary
0,468726,2016-01-02,2.673749e-11
1,134145,2016-01-02,1.069500e-12
2,66774,2016-01-02,2.228576e-12
3,178928,2016-01-02,8.730611e-11
4,679693,2016-01-02,3.492244e-11
...,...,...,...
52032794,636021,2023-02-23,4.365305e-12
52032795,1408806,2023-02-23,5.896654e-12
52032796,360238,2023-02-23,4.365305e-12
52032797,1007996,2023-02-23,6.547958e-13
